In [26]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re
import time
# Aqui comence con los cambios, quite algunas librerias que no se estan usando [GM]

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [27]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [28]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))
ahora_format = "%Y-%m-%d"'T'"%H:%M:%S"
ahora = str(now.strftime(ahora_format))
ahora_cdd = str(now.strftime("%Y-%m-%d"' '"%H:%M:%S"))
fechaAhora = str(now.strftime("%Y%m%d%H%M%S"))
datos_logs =""
#fechaAhora

### Definiendo indice principal con fecha de hoy
* 1.- Primer detalle (aqui se deberia de quitar esa fecha_hoy ? )
*     [ indice parameter= "dev-mintic-concat" ]   [GM]



In [31]:
indice = parametros.disponibilidad_tableros_index
indice_control = parametros.tableros_mintic_control

In [32]:
indice_control = 'prod-tableros_mintic_control'

### Función para generar JSON compatible con ES

In [33]:
def filterKeys(document, use_these_keys):
    return {key: document.get(key) for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [34]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
           "_source": ["gestion.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                              "field":"jerarquia_disponibilidad1"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['gestion.fechaControl']
except:
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-06-01T00:00:00'
print("ultima fecha para control de ejecucion gestion_estado_incidentes:",fecha_ejecucion)

ultima fecha para control de ejecucion gestion_estado_incidentes: 2021-12-01T12:50:00


### 1.- leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede.)

In [35]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
                           ,'nombreSede','latitud','longitud','id_Beneficiario','COD_ISO','codDanesede',
                           'cod_servicio','codDaneMuni','nombre_centro_pob','codCentroPoblado','codDaneInstitucionEdu',
                           'tipoSitio','detalleSitio','energia','region','matricula','DDA','grupoDesc','estadoInstalacion',
                           'nombreInstitucionEd']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except Exception as e:
    print(e)
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()

In [36]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'

In [37]:
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['gestion.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['gestion.location']=datos_semilla['gestion.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

In [38]:
# Impresión de dataframe AP 2021-09-16
datos_semilla

,nombreSede,DDA,estadoInstalacion,COD_ISO,energia,nombre_departamento,codCentroPoblado,nombreInstitucionEd,codDanesede,tipoSitio,...,matricula,energiadesc,grupoDesc,cod_servicio,region,detalleSitio,nombre_municipio,id_Beneficiario,codDaneInstitucionEdu,gestion.location
0,IE CURRULAO,5.0,1.0,CO-ANT,4.0,ANTIOQUIA,205837000310.0,IE CURRULAO,2.058370e+11,1.0,...,2177.0,OTROS,OBLIGATORIOS,,A,8.0,TURBO,21884,2.058370e+11,"7.93723286,-76.75345361"
1,CER LA PAULINA,4.0,4.0,CO-ANT,1.0,ANTIOQUIA,205854000501.0,C E R LA PAULINA,2.058540e+11,1.0,...,111.0,RED INTERCONECTADA,OBLIGATORIOS,ZGYO176,A,8.0,VALDIVIA,22154,2.058540e+11,"6.7048548,-76.14525469"
2,CENTRO EDUCATIVO RURAL EL CERRO,4.0,4.0,CO-ANT,1.0,ANTIOQUIA,105284000214.0,I E MANUEL ANTONIO TORO,2.052840e+11,1.0,...,22.0,RED INTERCONECTADA,OBLIGATORIOS,ZGYO177,A,8.0,FRONTINO,20022,1.052840e+11,"4.225166667,-73.10716667"
3,C E R MERCEDES YEPES,5.0,1.0,CO-ANT,4.0,ANTIOQUIA,205040000357.0,C E R EL CARMIN,2.050400e+11,1.0,...,42.0,OTROS,OBLIGATORIOS,ZCKE248,A,8.0,ANORI,18787,2.050400e+11,"6.93848274,-75.15117719"
4,C E R MEDIAS FALDAS,5.0,1.0,CO-ANT,4.0,ANTIOQUIA,105040000212.0,I E ANORI,2.050400e+11,1.0,...,84.0,OTROS,OPCION 2,ZIFV658,A,8.0,ANORI,18802,1.050400e+11,"7.04797951,-75.19442611"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,SANTA RITA,4.0,4.0,CO-ATL,1.0,ATLANTICO,108560000303.0,IE TECNICA AGROPECUARIA LA CANDELARIA,2.086060e+11,1.0,...,119.0,RED INTERCONECTADA,OBLIGATORIOS,ZZZY418,A,8.0,PONEDERA,22521,2.086060e+11,"10.50708786,-74.8144936"
9996,IE TECNICA AGROPECUARIA DE PUERTO GIRALDO - SE...,4.0,3.0,CO-ATL,3.0,ATLANTICO,208560000324.0,IE TECNICA AGROPECUARIA DE PUERTO GIRALDO,2.086380e+11,1.0,...,570.0,SOLUCION SOLAR,OBLIGATORIOS,ZZZY554,A,8.0,PONEDERA,22522,2.086380e+11,"10.99614816,-74.94199796"
9997,C E R ANTONIA SANTOS,5.0,1.0,CO-ANT,4.0,ANTIOQUIA,205890000356.0,I E R GUILLERMO AGUILAR,2.058900e+11,1.0,...,63.0,OTROS,OPCION 1,,A,8.0,YOLOMBO,22367,2.058900e+11,"6.95446002,-73.96020054"
9998,INSTITUCION EDUCATIVA TECNICO AGROPECUARIA DE ...,4.0,3.0,CO-ATL,3.0,ATLANTICO,208606000081.0,INSTITUCION EDUCATIVA TECNICO AGROPECUARIA DE ...,2.086850e+11,1.0,...,747.0,SOLUCION SOLAR,OBLIGATORIOS,VZVF163,A,8.0,REPELON,22526,2.086850e+11,"10.57200145,-75.14836044"


### 2.- Leyendo datos de Cambium-Device-Devices

In [39]:
# Se extraen los datos de cambium-devicedevices,  'cambium-devicedevices'

def traerDDevices():
#     print(fecha_max)
#     print(fecha_tope)
    
    total_docs = 30000
    try:
        response = es.search(
            index= parametros.cambium_d_d_index,
            body={
                "_source": ["registration_date","mac","ip","ap_group","site_id","status","name", "@timestamp"]
                ,"query": {
                    "match_all": {}
                  }
            },
            size=total_docs
        )
        elastic_docs = response["hits"]["hits"]
        datos_DDevices = pd.DataFrame([x["_source"] for x in elastic_docs])
        
#         datos_DDevices = datos_DDevices[(datos_DDevices["registration_date"] >= fecha_max) & (datos_DDevices["registration_date"] < fecha_tope)]
#         print(datos_DDevices)
        
        datos_DDevices['site_id'] = datos_DDevices['site_id'].str.strip()
        datos_DDevices  = datos_DDevices.rename(columns={'ap_group':'gestion.ptos_acceso'})
        datos_DDevices  = datos_DDevices.dropna(subset=['site_id'])
        datos_DDevices.fillna('', inplace=True)
        datos_DDevices = datos_DDevices.drop(datos_DDevices[(datos_DDevices['site_id']=='')].index)
        datos_DDevices.sort_values(['site_id','gestion.ptos_acceso'], inplace=True)
        
        datos_DDevices['gestion.ptos_acceso'] = datos_DDevices['gestion.ptos_acceso'].str.split("-", n = 1, expand = True)[0]
        datos_DDevices['gestion.ptos_acceso'] = datos_DDevices['gestion.ptos_acceso'].str.split("_", n = 1, expand = True)[0]
        datos_DDevices['gestion.ptos_acceso'] = datos_DDevices['gestion.ptos_acceso'].str.split(".", n = 1, expand = True)[0]
        datos_DDevices = datos_DDevices.drop(datos_DDevices[(datos_DDevices['gestion.ptos_acceso']=='')].index)
        datos_DDevices = datos_DDevices.drop_duplicates('mac')
        
        return datos_DDevices 
    except Exception as e:
        print(e)
        return pd.DataFrame()
    

Realizando bucle hasta conseguir datos de servicemanager-incidentes o hasta la fecha actual para realizar la carga de datos

In [40]:
def drop_invalid_values(df, fields=[], values = ['NA','',np.NaN,None],operator='or'):
    fields = df.columns if fields==[] else fields
    if operator=='or':
        freduce = np.logical_or.reduce
    elif operator=='and':
        freduce = np.logical_and.reduce
    else:
        return
    
    df.drop(df[freduce([df[c].isin(values) for c in fields])].index,inplace=True)

In [41]:
datos_CDD=pd.DataFrame(columns=['registration_date', 
                                    'gestion.ptos_acceso', 
                                    'ip', 
                                    'site_id', 
                                    'mac',
                                    'status',
                                    '@timestamp'])

In [42]:
# fecha_max_mintic = fecha_ejecucion.replace('T',' ')

# fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d %H:%M:%S")+timedelta(minutes=60)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
# datos_CDD = traerDDevices(fecha_max_mintic,fecha_tope_mintic)
datos_CDD = traerDDevices()
#datos_CDD = traerDDevices(fecha_max_mintic,fecha_tope_mintic)

# if datos_CDD is None or datos_CDD.empty:
#     while (datos_CDD is None or datos_CDD.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S.%f"))):
#         fecha_max_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d %H:%M:%S")+timedelta(minutes=60)).strftime("%Y-%m-%d %H:%M:%S")
#         fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, "%Y-%m-%d %H:%M:%S")+timedelta(minutes=60)).strftime("%Y-%m-%d %H:%M:%S")
#         datos_CDD = traerDDevices(fecha_max_mintic,fecha_tope_mintic)
# else:
#     pass

if datos_CDD.empty:
    datos_CDD=pd.DataFrame(columns=['registration_date', 
                                    'gestion.ptos_acceso', 
                                    'ip', 
                                    'site_id', 
                                    'mac',
                                    'status',
                                    '@timestamp'])


if datos_CDD is None or datos_CDD.empty:
    datos_logs = datos_logs +"\n No trajo datos en este rango de fecha. "
    datos_CDD2=pd.DataFrame(columns=['site_id', 'status'])
else:
    drop_invalid_values(datos_CDD,['gestion.ptos_acceso','site_id','status'])
    
    datos_CDD  = datos_CDD.dropna(subset=['site_id'])

    datos_CDD = datos_CDD.drop(datos_CDD[(datos_CDD['site_id']=='')].index)
    datos_logs= datos_logs + "\n total reg.: " + str(datos_CDD["site_id"].size) + "    viene con (registration_date,gestion.ptos_acceso,ip,site_id,mac,status)"

    #datos_CDD2 = datos_CDD[["site_id","registration_date", "status", "gestion.ptos_acceso"]]
    datos_CDD2 = datos_CDD[["site_id", "status"]]    #datos_CDD2 = datos_CDD2[["site_id","registration_date", "status", "gestion.ptos_acceso"]].groupby(["site_id", "status", "gestion.ptos_acceso"]).agg(['max']).reset_index()
    datos_CDD2 = datos_CDD2[["site_id","status"]].groupby(["site_id"]).agg(['count']).reset_index()

    datos_CDD2.columns = datos_CDD2.columns.droplevel(1)       
    datos_logs = datos_logs +"\n se quitan los repetidos status=(offline, online, onboaring) total reg.: " + str(datos_CDD2["site_id"].size)
 
#print (datos_logs)    # ojoooooo  cuando se pase a Produccion se quita esta linea

In [43]:
datos_CDD.rename(columns={'@timestamp':'registration_date1'},inplace=True)

In [44]:
if datos_CDD.empty:
    datos_CDD=pd.DataFrame(columns=['registration_date', 
                                    'gestion.ptos_acceso',
                                    'ip', 
                                    'site_id', 
                                    'mac',
                                    'status',
                                    'registration_date1'])

In [45]:
def disponibilidad(x):
    resp = 'No disponible'
    if 'online' in list(x):
        resp='Disponible'
    return resp

df_dispo = pd.concat([datos_CDD.groupby(['site_id'])['status'].apply(lambda x: disponibilidad(x)),
                      datos_CDD.groupby(['site_id']).registration_date.max('registration_date')],
                      axis=1)


df_dispo.rename(columns={'status':'disponibilidad','registration_date':'max_fecha'},inplace=True)

df_dispo.reset_index(inplace=True)


df_dispo= pd.merge(df_dispo,datos_CDD[['site_id','mac','ip','gestion.ptos_acceso','status']],on=['site_id'],how='inner')

df_dispo.rename(columns={'status':'status.macRed','gestion.ptos_acceso':'ap_group'},inplace=True)

In [46]:
df_dispo.columns

Index(['site_id', 'disponibilidad', 'max_fecha', 'mac', 'ip', 'ap_group',
       'status.macRed'],
      dtype='object')

Se limpian datos mal formados de ap_group

In [47]:
if not df_dispo.empty:
    df_dispo['ap_group'] = df_dispo['ap_group'].str.split("-", n = 1, expand = True)[0]
    df_dispo['ap_group'] = df_dispo['ap_group'].str.split("_", n = 1, expand = True)[0]
    df_dispo['ap_group'] = df_dispo['ap_group'].str.split(".", n = 1, expand = True)[0]
    df_dispo = df_dispo.drop(df_dispo[(df_dispo['ap_group']=='')].index)
    df_dispo = df_dispo.drop(df_dispo[(df_dispo['ap_group']=='PRUEBA OUTDOOR')].index)

In [48]:
df_dispo

,site_id,disponibilidad,max_fecha,mac,ip,ap_group,status.macRed
0,10003700,Disponible,2021-07-04 18:03:10,BC:E6:7C:5D:20:E1,172.25.41.219,INDOOR,online
1,10003700,Disponible,2021-07-04 18:03:10,BC:A9:93:0E:2F:15,172.25.41.220,OUTDOOR,online
2,10003700,Disponible,2021-07-04 18:03:10,BC:A9:93:0E:63:3E,172.25.41.221,OUTDOOR,online
3,10501-ZZZY338,Disponible,2021-07-15 12:05:14,BC:E6:7C:ED:8A:FE,172.25.46.251,INDOOR,online
4,10501-ZZZY338,Disponible,2021-07-15 12:05:14,BC:E6:7C:E8:3F:AE,172.25.46.253,OUTDOOR,online
...,...,...,...,...,...,...,...
9763,LABORATORIO PEREIRA,No disponible,2021-07-08 12:58:57,BC:A9:93:0C:7B:35,172.25.132.68,OUTDOOR,offline
9764,LABORATORIO PEREIRA,No disponible,2021-07-08 12:58:57,BC:A9:93:0C:AF:4F,172.25.132.69,OUTDOOR,offline
9765,PRUEBA-LABORATORIO,Disponible,2021-07-15 18:55:15,BC:E6:7C:EC:DD:45,172.25.132.75,INDOOR,online
9766,PRUEBA-LABORATORIO,Disponible,2021-07-15 18:55:15,BC:E6:7C:E7:FC:2B,172.25.132.77,OUTDOOR,online


## Lectura de estados
* estos estados guardan el ultimo estado "offline" en que estuvo el centro 


In [49]:
total_docs = 10000
try:
    response = es.search(
        index= "edo_sitio-" + indice,
        body={
               "_source": ['site_id','fechahora','itsm_incident_id']
        },
        size=total_docs
    )    
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    edo_sitio = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    
    if edo_sitio.empty:
        edo_sitio = pd.DataFrame(columns=['site_id','fechahora','itsm_incident_id'])

except Exception as e:
#     print(e.message)
    edo_sitio = pd.DataFrame(columns=['site_id','fechahora','itsm_incident_id'])
    
edo_sitio["fechahora"] = fechaAhora
#edo_sitio["itsm_incident_id"]    = "0"
  

In [50]:
edo_sitio

,fechahora,site_id,itsm_incident_id
0,20211201151023,26025-ZZZY170,0
1,20211201151023,26038-ZZZY293,0
2,20211201151023,31895-ZGYO272,0
3,20211201151023,31902-ZGYO433,0
4,20211201151023,68746-ZGYO443,0
...,...,...,...
739,20211201151023,44738-ZGYO662,0
740,20211201151023,44747-ZGYO413,0
741,20211201151023,45250-ZZZY586,0
742,20211201151023,45903-ZGYO225,0


In [51]:
print(datos_CDD2)
try:
    concat0 = pd.merge(datos_semilla, datos_CDD , on=['site_id'],how='inner')
except Exception as e:
#     print(e.message)
    concat0 = pd.DataFrame()

datos_logs = datos_logs +"\n se cruza con SEMILLA  total reg.: " + str(concat0["site_id"].size)

                  site_id  status
0                10003700       3
1           10501-ZZZY338       3
2           10537-ZCKE175       3
3           10786-ZGYO077       3
4           10835-ZGYO372       3
...                   ...     ...
3226        74587-ZIFV879       3
3227           777-PILOTO       3
3228        99999-ZGYO055       3
3229  LABORATORIO PEREIRA       3
3230   PRUEBA-LABORATORIO       3

[3231 rows x 2 columns]


In [55]:
datos_CDD2.to_excel("site_id_status.xlsx")

In [58]:
concat0

,nombreSede,DDA,estadoInstalacion,COD_ISO,energia,nombre_departamento,codCentroPoblado,nombreInstitucionEd,codDanesede,tipoSitio,...,codDaneInstitucionEdu,gestion.location,registration_date,registration_date1,gestion.ptos_acceso,ip,name,mac,status,cantidad
0,CER LA PAULINA,4.0,4.0,CO-ANT,1.0,ANTIOQUIA,205854000501.0,C E R LA PAULINA,2.058540e+11,1.0,...,2.058540e+11,"6.7048548,-76.14525469",2021-04-21 12:00:04,2021-12-01T15:03:46.797653,INDOOR,172.28.107.139,22154-ZGYO176-AP-INT,BC:E6:7C:5E:33:21,online,0
1,CER LA PAULINA,4.0,4.0,CO-ANT,1.0,ANTIOQUIA,205854000501.0,C E R LA PAULINA,2.058540e+11,1.0,...,2.058540e+11,"6.7048548,-76.14525469",2021-04-21 11:39:32,2021-12-01T15:03:46.797653,OUTDOOR,172.28.107.141,22154-ZGYO176-AP-EXT2,BC:E6:7C:4F:07:2F,online,0
2,CER LA PAULINA,4.0,4.0,CO-ANT,1.0,ANTIOQUIA,205854000501.0,C E R LA PAULINA,2.058540e+11,1.0,...,2.058540e+11,"6.7048548,-76.14525469",2021-04-21 12:02:38,2021-12-01T15:03:46.797653,OUTDOOR,172.28.107.140,22154-ZGYO176-AP-EXT1,BC:E6:7C:4F:47:2A,online,0
3,CER LA PAULINA,4.0,4.0,CO-ANT,1.0,ANTIOQUIA,205854000501.0,C E R LA PAULINA,2.058540e+11,1.0,...,2.058540e+11,"6.7048548,-76.14525469",2021-04-21 12:00:04,2021-12-01T15:03:46.797653,INDOOR,172.28.107.139,22154-ZGYO176-AP-INT,BC:E6:7C:5E:33:21,online,0
4,CER LA PAULINA,4.0,4.0,CO-ANT,1.0,ANTIOQUIA,205854000501.0,C E R LA PAULINA,2.058540e+11,1.0,...,2.058540e+11,"6.7048548,-76.14525469",2021-04-21 11:39:32,2021-12-01T15:03:46.797653,OUTDOOR,172.28.107.141,22154-ZGYO176-AP-EXT2,BC:E6:7C:4F:07:2F,online,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12554,IET LOS ALPES - SEDE PRINCIPAL,4.0,4.0,CO-TOL,1.0,TOLIMA,273873000615.0,INSTITUCION EDUCATIVA TECNICA LOS ALPES,2.735558e+11,1.0,...,2.735550e+11,"1.804166667,-75.50111111",2021-06-15 12:11:40,2021-12-01T15:03:46.797653,OUTDOOR,172.25.130.157,49603-ZZZY503-AP-EXT2,BC:A9:93:00:30:D1,offline,0
12555,IET LOS ALPES - SEDE PRINCIPAL,4.0,4.0,CO-TOL,1.0,TOLIMA,273873000615.0,INSTITUCION EDUCATIVA TECNICA LOS ALPES,2.735558e+11,1.0,...,2.735550e+11,"1.804166667,-75.50111111",2021-06-15 11:58:47,2021-12-01T15:03:46.797653,OUTDOOR,172.25.130.156,49603-ZZZY503-AP-EXT1,BC:A9:93:00:30:B8,offline,0
12556,BATALLON DE INSTRUCCION NO 6,5.0,1.0,CO-TOL,4.0,TOLIMA,333.0,BATALLON DE INSTRUCCION NO 6,9.990000e+02,2.0,...,9.990000e+02,"4.65092937,-74.106159",2021-09-02 17:36:48,2021-12-01T15:03:46.797653,INDOOR,172.25.177.123,74587-ZIFV879-AP-INT,BC:A9:93:10:53:4E,offline,0
12557,BATALLON DE INSTRUCCION NO 6,5.0,1.0,CO-TOL,4.0,TOLIMA,333.0,BATALLON DE INSTRUCCION NO 6,9.990000e+02,2.0,...,9.990000e+02,"4.65092937,-74.106159",2021-09-02 17:34:44,2021-12-01T15:03:46.797653,OUTDOOR,172.25.177.124,74587-ZIFV879-AP-EXT1,BC:A9:93:0C:79:AB,offline,0


In [57]:
datos_semilla

,nombreSede,DDA,estadoInstalacion,COD_ISO,energia,nombre_departamento,codCentroPoblado,nombreInstitucionEd,codDanesede,tipoSitio,...,matricula,energiadesc,grupoDesc,cod_servicio,region,detalleSitio,nombre_municipio,id_Beneficiario,codDaneInstitucionEdu,gestion.location
0,IE CURRULAO,5.0,1.0,CO-ANT,4.0,ANTIOQUIA,205837000310.0,IE CURRULAO,2.058370e+11,1.0,...,2177.0,OTROS,OBLIGATORIOS,,A,8.0,TURBO,21884,2.058370e+11,"7.93723286,-76.75345361"
1,CER LA PAULINA,4.0,4.0,CO-ANT,1.0,ANTIOQUIA,205854000501.0,C E R LA PAULINA,2.058540e+11,1.0,...,111.0,RED INTERCONECTADA,OBLIGATORIOS,ZGYO176,A,8.0,VALDIVIA,22154,2.058540e+11,"6.7048548,-76.14525469"
2,CENTRO EDUCATIVO RURAL EL CERRO,4.0,4.0,CO-ANT,1.0,ANTIOQUIA,105284000214.0,I E MANUEL ANTONIO TORO,2.052840e+11,1.0,...,22.0,RED INTERCONECTADA,OBLIGATORIOS,ZGYO177,A,8.0,FRONTINO,20022,1.052840e+11,"4.225166667,-73.10716667"
3,C E R MERCEDES YEPES,5.0,1.0,CO-ANT,4.0,ANTIOQUIA,205040000357.0,C E R EL CARMIN,2.050400e+11,1.0,...,42.0,OTROS,OBLIGATORIOS,ZCKE248,A,8.0,ANORI,18787,2.050400e+11,"6.93848274,-75.15117719"
4,C E R MEDIAS FALDAS,5.0,1.0,CO-ANT,4.0,ANTIOQUIA,105040000212.0,I E ANORI,2.050400e+11,1.0,...,84.0,OTROS,OPCION 2,ZIFV658,A,8.0,ANORI,18802,1.050400e+11,"7.04797951,-75.19442611"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,SANTA RITA,4.0,4.0,CO-ATL,1.0,ATLANTICO,108560000303.0,IE TECNICA AGROPECUARIA LA CANDELARIA,2.086060e+11,1.0,...,119.0,RED INTERCONECTADA,OBLIGATORIOS,ZZZY418,A,8.0,PONEDERA,22521,2.086060e+11,"10.50708786,-74.8144936"
9996,IE TECNICA AGROPECUARIA DE PUERTO GIRALDO - SE...,4.0,3.0,CO-ATL,3.0,ATLANTICO,208560000324.0,IE TECNICA AGROPECUARIA DE PUERTO GIRALDO,2.086380e+11,1.0,...,570.0,SOLUCION SOLAR,OBLIGATORIOS,ZZZY554,A,8.0,PONEDERA,22522,2.086380e+11,"10.99614816,-74.94199796"
9997,C E R ANTONIA SANTOS,5.0,1.0,CO-ANT,4.0,ANTIOQUIA,205890000356.0,I E R GUILLERMO AGUILAR,2.058900e+11,1.0,...,63.0,OTROS,OPCION 1,,A,8.0,YOLOMBO,22367,2.058900e+11,"6.95446002,-73.96020054"
9998,INSTITUCION EDUCATIVA TECNICO AGROPECUARIA DE ...,4.0,3.0,CO-ATL,3.0,ATLANTICO,208606000081.0,INSTITUCION EDUCATIVA TECNICO AGROPECUARIA DE ...,2.086850e+11,1.0,...,747.0,SOLUCION SOLAR,OBLIGATORIOS,VZVF163,A,8.0,REPELON,22526,2.086850e+11,"10.57200145,-75.14836044"


Haciendo merge entre semilla e incidentes

In [53]:
##esto es el deber ser, lo politicamente correcto 
if(concat0.empty):
    fecha_ejecucion = (datetime.strptime(fecha_ejecucion, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=60)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'

    if fecha_ejecucion > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
        fecha_ejecucion = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'

    response = es.index(
        index = indice_control,
        id = 'jerarquia_disponibilidad1',
        body = { 'jerarquia_disponibilidad1': 'jerarquia_disponibilidad1','gestion.fechaControl' : fecha_ejecucion}
    )
    print("actualizada fecha control de ejecucion:",fecha_ejecucion)
    exit()

result1=concat0
result2=concat0

result1['cantidad'] = 0
result2['cantidad'] = 0

result1 = result1.groupby(['site_id'])['site_id'].count().to_frame()
result2 = result2.groupby(['site_id', 'status'])['site_id'].count().to_frame()

result1 = result1.rename(columns={'site_id' :  'cantidad_x'})
result2 = result2.rename(columns={'site_id' :  'cantidad_y'})
result1 = result1.reset_index()
result2 = result2.reset_index()
#result1 = result1[['site_id',"cantidad"]].groupby(['site_id']).agg(['count'])
#result2 = result2[['site_id', 'status',"cantidad"]].groupby(['site_id', 'status']).agg(['count'])
#result1 = result1.columns.droplevel(1)
#result2 = result2.columns.droplevel(1)
#datos_CDD2[["site_id","status"]].groupby(["site_id"]).agg(['count']).reset_index()
result_completo = pd.merge(result1, result2,on=['site_id'],how='inner')
#result_completo["gestion.estadoCentro"] = result_completo.apply(lambda row: 'DESCONECTADOS' if( row.status == 'offline' and row.cantidad_x == row.cantidad_y) else 'CONECTADOS', axis=1)
result_completo["gestion.estadoCentro"] = result_completo.apply(lambda row: 'DESCONECTADOS' if( row.status  == 'offline' and row.cantidad_x == row.cantidad_y) else 'CONECTADOS', axis=1)

completo  = pd.merge(concat0, result_completo,on=['site_id'],how='inner')


#completo 
conectados = 0
desconectados = 0

bb = completo[(completo["gestion.estadoCentro"]=="DESCONECTADOS")].reset_index()
aa = completo[(completo["gestion.estadoCentro"]!="DESCONECTADOS")].reset_index()

conectados = aa["site_id"].size
desconectados = bb["site_id"].size
totales = desconectados + conectados
print (" Conectados: " , conectados , " desconectados: ", desconectados , " total: " , totales)
print(completo)
#i=0
#for index2 in todos_off:    
 #   print (i," reg. site_id: " )
 #   print (index2)
 #   i=i+1
    # df.loc[df['A'] > 2, 'B'] = new_val    ejemplo 

# if len(todos_off)>0:
#     print ("tienes registro .. todo_off")
# else:    
#     print ("NO  NOOOOO tienes registro .. todo_off")


 Conectados:  11195  desconectados:  2604  total:  13799
                           nombreSede  DDA  estadoInstalacion COD_ISO  \
0                      CER LA PAULINA  4.0                4.0  CO-ANT   
1                      CER LA PAULINA  4.0                4.0  CO-ANT   
2                      CER LA PAULINA  4.0                4.0  CO-ANT   
3                      CER LA PAULINA  4.0                4.0  CO-ANT   
4                      CER LA PAULINA  4.0                4.0  CO-ANT   
...                               ...  ...                ...     ...   
13794  IET LOS ALPES - SEDE PRINCIPAL  4.0                4.0  CO-TOL   
13795  IET LOS ALPES - SEDE PRINCIPAL  4.0                4.0  CO-TOL   
13796    BATALLON DE INSTRUCCION NO 6  5.0                1.0  CO-TOL   
13797    BATALLON DE INSTRUCCION NO 6  5.0                1.0  CO-TOL   
13798    BATALLON DE INSTRUCCION NO 6  5.0                1.0  CO-TOL   

       energia nombre_departamento codCentroPoblado  \
0          

### Realizando inserción en ultimo estado del sitio

In [751]:
#use_these_keys = ['site_id','gestion.estadoCentro','fechahora', 'itsm_incident_id']
#def doc_generator(df,use_these_keys):
#    df_iter = df.iterrows()
#    for index, document in df_iter:
#        yield {
#                "_index": "edo_sitio-" + indice, 
#                "_id": f"{'Estado-'+str(document['site_id'])}",
#                "_source": filterKeys(document,use_these_keys),
#            }
#if len(todos_off)>0:        
#    salida = helpers.bulk(es, doc_generator(todos_off,use_these_keys))
#    print("Fecha: ", now,"- Ult.Estado de los Centros Desconectados  insertadas en indice edo_sitio:",salida[0])


In [752]:
# es.indices.delete(index="edo_sitio-" + parametros.mintic_concat_index, ignore=[400, 404])
# aqui elimino todo el indice para poder ingresar los nuevos registros con los campos nuevos  GM

## 10.- insercion en el indice


In [753]:
# primero INSERTAR FINAL
completo.fillna({'fechahora':fechaAhora},inplace=True)
completo.fillna({'gestion.estadoCentro':'CONECTADOS'},inplace=True)
completo.fillna({'itsm_incident_id':0},inplace=True)

try:
    completo = completo.rename(columns={'id_Beneficiario' :  'gestion.estado.id_Beneficiario'
                                                                ,'nombreSede':'gestion.estado.nombreSede'
                                                                ,'site_id':'gestion.estado.site_id'
                                                                ,'nombre_departamento':'gestion.estado.dptoGestion'
                                                                ,'nombre_municipio':'gestion.estado.muniGestion'
                                                                ,'nombre_centro_pob':'gestion.estado.nombre_centro_pob'
                                                                ,'COD_ISO':'gestion.estado.COD_ISO'
                                                                ,'codDanesede':'gestion.estado.codDanesede'
                                                                ,'energiadesc':'gestion.estado.energiadesc'
                                                                ,'DDA':'gestion.estado.DDA'
                                                                ,'gestion.location':'gestion.estado.location'
                                                                ,'estadoInstalacion':'gestion.estado.estadoInstalacion'
                                                                ,'nombreInstitucionEd':'gestion.estado.nombreInstitucionEd'
                                                                ,'matricula':'gestion.estado.matricula'
                                                                ,'municipioPDET':'gestion.estado.municipioPDET'    
                                        ,'cod_servicio':'gestion.estado.cod_servicio'
                                        ,'codDaneMuni':'gestion.estado.codDaneMuni'
                                        ,'codCentroPoblado':'gestion.estado.codCentroPoblado'
                                        ,'codDaneInstitucionEdu':'gestion.estado.codDaneInstitucionEdu'
                                        ,'tipoSitio':'gestion.estado.tipoSitio'
                                        ,'detalleSitio':'gestion.estado.detalleSitio'
                                        ,'energia':'gestion.estado.energia'
                                        ,'region':'gestion.estado.region'
                                        ,'grupoDesc' :'gestion.estado.grupoDesc'
                                        })
                
    completo["gestion.fechaControl"] =  ahora_cdd
    completo["gestion.fecha"]=  completo["gestion.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    completo["gestion.hora"]=completo["gestion.fechaControl"].str.split(" ", n=1, expand=True)[1].str.split(":", n = 2, expand = True)[0]
    completo["gestion.minuto"]=completo["gestion.fechaControl"].str.split(" ", n=1, expand=True)[1].str.split(":", n = 2, expand = True)[1]
    completo["gestion.anyo"] =  completo["gestion.fecha"].str[0:4]
    completo["gestion.mes"]  =  completo["gestion.fecha"].str[5:7]
    completo["gestion.dia"]  =  completo["gestion.fecha"].str[8:10]
    completo["gestion.totales.cantDev"]  = totales  
    completo["gestion.totales.cantDevConectados"]  =  conectados
    completo["gestion.totales.cantDevdesConectados"]  =  desconectados
    
    print(completo)
except Exception as e:
    completo = pd.DataFrame(columns=['gestion.estado.nombreSede', 'gestion.estado.DDA','gestion.estado.estadoInstalacion', 'gestion.estado.COD_ISO',
   'gestion.estado.energia', 'gestion.estado.dptoGestion','gestion.estado.codCentroPoblado', 'gestion.estado.codDanesede',
   'gestion.estado.tipoSitio', 'gestion.estado.codDaneMuni','gestion.estado.nombre_centro_pob', 'gestion.estado.site_id',
   'gestion.estado.matricula', 'gestion.estado.energiadesc','gestion.estado.grupoDesc', 'gestion.estado.cod_servicio',
   'gestion.estado.region', 'gestion.estado.detalleSitio','gestion.estado.muniGestion', 'gestion.estado.id_Beneficiario',
   'gestion.estado.codDaneInstitucionEdu', 'gestion.estado.location','status', 'gestion.estadoCentro', 'itsm_incident_id', 'fechahora',
   'gestion.fechaControl', 'gestion.fecha', 'gestion.hora', 'gestion.anyo','gestion.mes', 'gestion.dia', 'gestion.totales.cantDev',
   'gestion.totales.cantDevConectados','gestion.totales.cantDevdesConectados'])
    print(e)


              gestion.estado.nombreSede  gestion.estado.DDA  \
0                      C.E.R LA PAULINA                   5   
1                      C.E.R LA PAULINA                   5   
2                      C.E.R LA PAULINA                   5   
3       CENTRO EDUCATIVO RURAL EL CERRO                   5   
4       CENTRO EDUCATIVO RURAL EL CERRO                   5   
...                                 ...                 ...   
5005                          EL DIVISO                   5   
5006                          EL DIVISO                   5   
5007  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   
5008  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   
5009  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   

      gestion.estado.estadoInstalacion gestion.estado.COD_ISO  \
0                                    3                 CO-ANT   
1                                    3                 CO-ANT   
2                                    3          

In [754]:
df_dispo.rename(columns={'site_id':'gestion.estado.site_id',
                         'disponibilidad' : 'gestion.estado.disponibilidad',
                         'max_fecha':'gestion.estado.max_fecha'},
                inplace=True)

idx=completo[completo['gestion.estado.matricula']=='No aplica'].index

completo.loc[idx,'gestion.estado.matricula']=0

print(df_dispo.columns)
completo = pd.merge(completo, df_dispo,on=['gestion.estado.site_id','mac'],how='inner')
print(completo)

Index(['gestion.estado.site_id', 'gestion.estado.disponibilidad',
       'gestion.estado.max_fecha', 'mac', 'ip', 'ap_group', 'status.macRed'],
      dtype='object')
              gestion.estado.nombreSede  gestion.estado.DDA  \
0                      C.E.R LA PAULINA                   5   
1                      C.E.R LA PAULINA                   5   
2                      C.E.R LA PAULINA                   5   
3       CENTRO EDUCATIVO RURAL EL CERRO                   5   
4       CENTRO EDUCATIVO RURAL EL CERRO                   5   
...                                 ...                 ...   
5005                          EL DIVISO                   5   
5006                          EL DIVISO                   5   
5007  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   
5008  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   
5009  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   

      gestion.estado.estadoInstalacion gestion.estado.COD_ISO  \
0           

In [755]:
completo.columns

Index(['gestion.estado.nombreSede', 'gestion.estado.DDA',
       'gestion.estado.estadoInstalacion', 'gestion.estado.COD_ISO',
       'gestion.estado.energia', 'gestion.estado.dptoGestion',
       'gestion.estado.codCentroPoblado', 'gestion.estado.nombreInstitucionEd',
       'gestion.estado.codDanesede', 'gestion.estado.tipoSitio',
       'gestion.estado.codDaneMuni', 'gestion.estado.nombre_centro_pob',
       'gestion.estado.site_id', 'gestion.estado.matricula',
       'gestion.estado.energiadesc', 'gestion.estado.grupoDesc',
       'gestion.estado.cod_servicio', 'gestion.estado.region',
       'gestion.estado.detalleSitio', 'gestion.estado.muniGestion',
       'gestion.estado.id_Beneficiario',
       'gestion.estado.codDaneInstitucionEdu', 'gestion.estado.location',
       'registration_date', 'registration_date1', 'gestion.ptos_acceso',
       'ip_x', 'name', 'mac', 'status_x', 'cantidad', 'cantidad_x', 'status_y',
       'cantidad_y', 'gestion.estadoCentro', 'gestion.fechaControl'

In [756]:
print(completo)
if not completo.empty:
    completo['@timestamp'] = now.isoformat()
    #completo["gestion.fechaControl"] =  completo['gestion.estado.max_fecha']
    completo["gestion.fechaControl"]  = now.isoformat().replace( 'T', ' ')
    completo["gestion.fecha"]=  completo["gestion.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    completo["gestion.hora"]=completo["gestion.fechaControl"].str.split(" ", n=1, expand=True)[1].str.split(":", n = 2, expand = True)[0]
    completo["gestion.minuto"]=completo["gestion.fechaControl"].str.split(" ", n=1, expand=True)[1].str.split(":", n = 2, expand = True)[1]
    completo["gestion.anyo"] =  completo["gestion.fecha"].str[0:4]
    completo["gestion.mes"]  =  completo["gestion.fecha"].str[5:7]
    completo["gestion.dia"]  =  completo["gestion.fecha"].str[8:10]

              gestion.estado.nombreSede  gestion.estado.DDA  \
0                      C.E.R LA PAULINA                   5   
1                      C.E.R LA PAULINA                   5   
2                      C.E.R LA PAULINA                   5   
3       CENTRO EDUCATIVO RURAL EL CERRO                   5   
4       CENTRO EDUCATIVO RURAL EL CERRO                   5   
...                                 ...                 ...   
5005                          EL DIVISO                   5   
5006                          EL DIVISO                   5   
5007  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   
5008  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   
5009  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   

      gestion.estado.estadoInstalacion gestion.estado.COD_ISO  \
0                                    3                 CO-ANT   
1                                    3                 CO-ANT   
2                                    3          

In [757]:
# insertando los datos en el indice 

use_these_keys = ['gestion.estado.id_Beneficiario'
                  ,'gestion.estadoCentro'
                  ,'gestion.estado.nombreSede'
                  ,'gestion.estado.COD_ISO'
                  ,'gestion.estado.codDanesede'
                  ,'gestion.estado.nombre_centro_pob'
                  ,'gestion.estado.site_id'
                  ,'gestion.estado.dptoGestion'
                  ,'gestion.estado.energiadesc'
                  ,'gestion.estado.muniGestion'
                  ,'gestion.estado.location'  
                  ,'gestion.estado.DDA'
                  ,'gestion.estado.nombreInstitucionEd'
                  ,'gestion.estado.matricula'
                  ,'gestion.estado.municipioPDET'    
                  ,'gestion.fechaControl'
                  ,'gestion.fecha'
                  ,'gestion.anyo'
                  ,'gestion.mes'
                  ,'gestion.dia'
                  ,'gestion.hora'
                  ,'gestion.totales.cantDev'
                  ,'gestion.totales.cantDevConectados'
                  ,'gestion.totales.cantDevdesConectados'
                  ,'gestion.estado.disponibilidad'
                  ,'gestion.estado.max_fecha'
                  ,'gestion.estado.cod_servicio'
                  ,'gestion.estado.codDaneMuni'                  
                  ,'gestion.estado.codCentroPoblado'
                  ,'gestion.estado.codDaneInstitucionEdu'
                  ,'gestion.estado.tipoSitio'
                  ,'gestion.estado.detalleSitio'
                  ,'gestion.estado.energia'
                  ,'gestion.estado.region'
                  ,'gestion.estado.grupoDesc'
                  ,'gestion.estado.estadoInstalacion'
                  ,'@timestamp'
                  ,'mac'
                  ,'ip'
                  ,'ap_group'
                  ,'status.macRed'
                  ,'gestion.minuto']                         

In [758]:
def doc_generator2(df,use_these_keys):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
#                 "_id": f"{'Estado-'+str(+document['gestion.estado.id_Beneficiario']) + '-' + str(document['gestion.fechaControl'])}",
                "_id": f"{str(document['mac']) + '-' + str(document['@timestamp'])}",
                "_source": filterKeys(document,use_these_keys),
            }

salida = helpers.bulk(es, doc_generator2(completo,use_these_keys))
print(completo)
print("Fecha: ", now,"- Llamadas insertadas en indice principal:",salida[0])

              gestion.estado.nombreSede  gestion.estado.DDA  \
0                      C.E.R LA PAULINA                   5   
1                      C.E.R LA PAULINA                   5   
2                      C.E.R LA PAULINA                   5   
3       CENTRO EDUCATIVO RURAL EL CERRO                   5   
4       CENTRO EDUCATIVO RURAL EL CERRO                   5   
...                                 ...                 ...   
5005                          EL DIVISO                   5   
5006                          EL DIVISO                   5   
5007  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   
5008  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   
5009  I.E.T. LOS ALPES - SEDE PRINCIPAL                   5   

      gestion.estado.estadoInstalacion gestion.estado.COD_ISO  \
0                                    3                 CO-ANT   
1                                    3                 CO-ANT   
2                                    3          

Actualizando fecha de control de ejecución

In [759]:
fecha_ejecucion = (datetime.strptime(fecha_ejecucion, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=60)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'

response = es.index(
        index = indice_control,
        id = 'jerarquia_disponibilidad1',
        body = { 'jerarquia_disponibilidad1': 'jerarquia_disponibilidad1','gestion.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-06-04T03:00:00
